In [1]:
import tensorflow as tf

/home/etudiant/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# DELTA_DIST+H - Recherche des hyperparamètres

Le modèle DELTA_DIST+H est le premier que l'on va entraîner. L'objectif est d'obtenir un modèle capable d'optimiser des molécules à partir des coordonnées non optimisées des atomes. La méthode d'entraînement est la suivante : on part des coordonnées optimisées des atomes de la molécule, on leur ajoute un bruit gaussien, on calcule les distances des atomes bruités par rapport aux atomes fictifs du repère, puis on tente d'apprendre au modèle à prédire la différence entre les distances des atomes bruités et les distances des atomes optimisés.

Dans ce notebook on propose une solution pour faire varier les hyperparamètres du modèle et chercher leurs meilleures valeurs pour entraîner ensuite un modèle possédant les hyperparamètres produisant les meilleurs résultats. 

## Création du RN

### Fonctions de coût et d'évaluation du modèle

#### Fonction renvoyant le masque des atomes à prendre en compte pour les calculs

L'entrée et la sortie du RN étant définies par une méthode de padding, seul un certain nombre d'entrées et de sortie est utilisé pour chaque exemple d'entraînement en fonction du nombre d'atomes de la molécule. On définit ici une fonction qui renvoie le masque des différences de distances à prendre en compte sur les données en entrée et les étiquettes.

In [4]:
import tensorflow as tf

def calcul_masque_atomes_definis(targets):
    """ On calcule le masque booléen des atomes donnés en entrée du RN en fonction du vecteur targets"""
    
    # On cherche à obtenir un masque booléen des atomes définis en entrée. Pour cela, on prend en entrée
    # les étiquettes sous la forme d'une matrice (200, 4) dont chaque ligne i est la distance de l'atome i avec
    # les atomes fictifs du repère. L'atome est indéfini ssi. la somme de la ligne est nulle. En effet,
    # un atome défini ne peut pas avoir une distance nulle avec les quatre atomes fictifs, et on veille
    # à ce que le vecteurs targets ne contienne que des valeurs nulles pour les atomes non définis.
    # On obtient donc un masque booléen de tous les atomes définis en entrée
    
    ## On somme les distances de chaque atome ##
    targets_dists_sums = tf.reduce_sum(targets, 1)
    
    ## On créé le masque des sommes différentes de zéro ##
    
    # Création des matrice de True et de False de la dimension de la matrice des sommes (nécessaires
    # pour tf.where)
    zeros = tf.cast(tf.zeros_like(targets_dists_sums),dtype=tf.bool)
    ones = tf.cast(tf.ones_like(targets_dists_sums),dtype=tf.bool)
    
    return tf.where(targets_dists_sums>0, ones, zeros)


#### Fonction de coût

In [5]:
def partial_rmse(predictions, targets):
    """ Calcule le RMSE partiel des prédictions par rapport aux valeurs attendues. Le RMSE est partiel car
    on ne le calcule que pour les sorties correspondant aux atomes donnés en entrée. En d'autres
    termes, on ne pousse pas le modèle à donner des distances nulles pour les atomes indéfinis
    en entrée"""
    

    # On met les prédictions et les cibles sous la forme d'une matrice (200, 4)
    predictions = tf.reshape(predictions, [-1, 4])
    targets = tf.reshape(targets, [-1, 4])
    
    # On calcule le masque des atomes définis selon les cibles
    defined_atoms_mask = calcul_masque_atomes_definis(targets)
    
    
    # On masque les prédictions et les étiquettes selon le masque des atomes définis
    targets_masked = tf.boolean_mask(targets, defined_mask)
    predictions_masked = tf.boolean_mask(predictions, defined_mask)
    
    
    return tf.sqrt(tf.reduce_mean(tf.squared_difference(predictions_masked, targets_masked)), name="rmse")
    

#### Fonction d'évaluation des performances (score R2)

In [6]:
def partial_r2_score(predictions, targets, inputs):
    """ Renvoie le score R2 de la prédiction (le calcul est effectué uniquement sur les résultats
    des atomes donnés en entrée) """
    
    # On met les prédictions et les cibles sous la forme d'une matrice (200, 4)
    predictions = tf.reshape(predictions, [-1, 4])
    targets = tf.reshape(targets, [-1, 4])
    
    # On calcule le masque des atomes définis selon les cibles
    defined_atoms_mask = calcul_masque_atomes_definis(targets)
    
    # Calcul de l'erreur totale
    total_error = tf.reduce_sum(tf.square(tf.subtract(targets, tf.reduce_mean(targets))))
                
    # Calcul de l'erreur inexpliquée
    unexplained_error = tf.reduce_sum(tf.square(tf.subtract(targets, predictions)))

    init_op = tf.initialize_all_variables()
    
    r2 = tf.subtract(1.0, tf.divide(unexplained_error, total_error), "r2_score")

#### Définition d'une fonction créant le RN

In [7]:
def creer_fc3_1k_15k_800(dropout=0.95, stddev_init=0.001, learning_rate=0.05):
    """ Fonction créant un réseau de neurones de type fully connected, ayant une couche d'entrée de 1000
    neurones, une couche cachée de 15000 neurones une sortie de 800 neurones (en plus d'une couche 
    d'un neurone de coût)
    Inputs 
    dropout : proba que chaque neurone ne soit pas désactivé
    stddev_init : sigma de l'initialisation des poids
    learning_rate : taux d'apprentissage
    """

    # On créé l'initialisateur de tenseur avec une loi normale tronquée. sigma = stddev_init, et les 
    # valeurs à plus de 2sigma sont re-tirées
    winit = tfl.initializations.truncated_normal(stddev=stddev_init, dtype=tf.float32, seed=None)

    # On créé l'input du RN
    network = input_data(shape=[None, 1000], name='input')

    # On créé un tenseur (network) à partir des inputs (var network) et de taille de sortie 15000 -> couche 
    # centrale
    # On utilise une fonction d'activation relu : max(features, 0)
    # On utilise l'initialisateur avec la loi normale tronquée
    network = fully_connected(network, 15000, activation='relu', name='fc0', weights_init=winit)
    
    # On détruit des neurones aléatoirement avec une probabilité donnée en entrée
    network = dropout(network, dropout)

    # On ajoute la couche de sortie du réseau
    # Fonction d'activation prelu
    # Initilisée avec la loi normale tronquée
    network = fully_connected(network, 800, activation='prelu', name='outlayer', weights_init=winit)

    # Couche d'évaluation du modèle. Utilisation d'une descente stochastique Adam
    # Learning rate = 0.05
    # Loss = fonction définie rmse
    network = regression(network, optimizer='adam', learning_rate=0.05,
        loss=partial_rmse, metric=partial_r2_score, name='target')
    
    return network

## Préparation des données

#### Fonction renvoyant deux sous-ensembles du jeu d'entrainement : un jeu d'entraînement et un jeu de validation

In [ ]:
def get_fold(train_set, targets, reduce_train_fold_proportion):
    """ Permet d'obtenir deux sous-ensembles du jeu d'entraînement afin de ne pas travailler sur le jeu
    d'entraînement total pour la recherche par quadrillage et donc de gagner du temps d'exécution"""
    
    
    
    